In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import optuna
import shap

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# 데이터 로드
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv("./test.csv")
df_sample_submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# ID 제거
df_train.drop(columns=['ID'], inplace=True)
df_test_ids = df_test['ID']
df_test.drop(columns=['ID'], inplace=True)

In [ ]:
# 타겟 변수 분리
X = df_train.drop(columns=['임신 성공 여부'], errors='ignore').copy()
y = df_train['임신 성공 여부']

df_test = df_test.drop(columns=['임신 성공 여부'], errors='ignore').copy()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
import shap

# 유니크 값이 1개인 컬럼 삭제
drop_cols = ['불임 원인 - 여성 요인', '난자 채취 경과일']
X.drop(columns=drop_cols, inplace=True, errors='ignore')
df_test.drop(columns=drop_cols, inplace=True, errors='ignore')

# '배란 자극 여부'에 따른 '배란 유도 유형' 수정
if '배란 유도 유형' in X.columns:
    X.loc[(X['배란 자극 여부'] == 0) & (X['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '시행 없음'
    X.loc[(X['배란 자극 여부'] == 1) & (X['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '기록되지 않은 시행'

if '배란 유도 유형' in df_test.columns:
    df_test.loc[(df_test['배란 자극 여부'] == 0) & (df_test['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '시행 없음'
    df_test.loc[(df_test['배란 자극 여부'] == 1) & (df_test['배란 유도 유형'] == '알 수 없음'), '배란 유도 유형'] = '기록되지 않은 시행'

# 숫자형 컬럼 값 정수 변환
cols_to_slice = ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수',
                 '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수']
for col in cols_to_slice:
    X[col] = X[col].astype(str).str.extract(r'(\d+)').astype(float).fillna(0).astype(int)
    df_test[col] = df_test[col].astype(str).str.extract(r'(\d+)').astype(float).fillna(0).astype(int)

# 특정 NaN 값 채우기 (훈련 데이터에서 가장 많이 등장한 값 적용)
columns_to_fill_most_frequent = ['특정 시술 유형', '배아 생성 주요 이유']
imputer_cat = SimpleImputer(strategy="most_frequent")
X[columns_to_fill_most_frequent] = imputer_cat.fit_transform(X[columns_to_fill_most_frequent])
df_test[columns_to_fill_most_frequent] = imputer_cat.transform(df_test[columns_to_fill_most_frequent])

# 특정 값이 NaN이면 0으로 변환
columns_to_fill_zero = ['단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '이식된 배아 수']
for col in columns_to_fill_zero:
    X[col] = X[col].fillna(0)
    df_test[col] = df_test[col].fillna(0)

# 결측치 처리 (수치형 컬럼 → 평균값으로 대체)
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
imputer_num = SimpleImputer(strategy="mean")

X[numeric_features] = imputer_num.fit_transform(X[numeric_features])
df_test[numeric_features] = imputer_num.transform(df_test[numeric_features])

In [ ]:
# 결측치 처리 (범주형 컬럼 → 최빈값으로 대체)
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
imputer_cat = SimpleImputer(strategy="most_frequent")

X[categorical_features] = imputer_cat.fit_transform(X[categorical_features])
df_test[categorical_features] = imputer_cat.transform(df_test[categorical_features])

# Feature Engineering (결측치 여부 Feature 추가)
for col in numeric_features:
    X[f'{col}_missing'] = X[col].isna().astype(int)
    df_test[f'{col}_missing'] = df_test[col].isna().astype(int)

# 로그 변환 적용 (필요한 변수에만)
#log_transform_cols = ['총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수']
#for col in log_transform_cols:
 #   X[col] = np.log1p(X[col])
   # df_test[col] = np.log1p(df_test[col])

# 범주형 변수 인코딩 (OrdinalEncoder 사용)
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X[categorical_features] = encoder.fit_transform(X[categorical_features])
df_test[categorical_features] = encoder.transform(df_test[categorical_features])

# Feature Scaling
scaler = StandardScaler()
X[numeric_features] = scaler.fit_transform(X[numeric_features])
df_test[numeric_features] = scaler.transform(df_test[numeric_features])

# Feature Selection (LightGBM Feature Importance 기반)
lgbm = LGBMClassifier(n_estimators=200, random_state=42, n_jobs=-1)
lgbm.fit(X, y)

# Feature Importance 기반으로 중요도가 0.001 이상인 변수만 선택
feature_importance = pd.Series(lgbm.feature_importances_, index=X.columns).sort_values(ascending=False)
selected_features = feature_importance[feature_importance > 0.001].index.tolist()

# 선택된 Feature만 유지
X = X[selected_features]
df_test = df_test[selected_features]

print("전처리 최적화 완료!")

<ipython-input-38-a7ce4931b3b4>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f'{col}_missing'] = X[col].isna().astype(int)
<ipython-input-38-a7ce4931b3b4>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[f'{col}_missing'] = df_test[col].isna().astype(int)
<ipython-input-38-a7ce4931b3b4>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.096421 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 779
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
✅ 전처리 최적화 완료!


In [ ]:
# 훈련 / 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 800),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
        'num_leaves': trial.suggest_int('num_leaves', 20, 80),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 10.0),
        'random_state': 42,
        'n_jobs': -1
    }
    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_val_pred = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, y_val_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
best_params = study.best_params
print(f"🔹 Best Params: {best_params}")

[I 2025-02-14 08:24:48,450] A new study created in memory with name: no-name-febf248c-a784-458b-9a65-5ec273fb983f
<ipython-input-43-980fb401a53f>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
<ipython-input-43-980fb401a53f>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10.0),
<ipython-input-43-980fb401a53f>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_lambda': trial.s

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.085039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 772
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[I 2025-02-14 08:25:05,831] Trial 0 finished with value: 0.736593602880274 and parameters: {'n_estimators': 445, 'learning_rate': 0.010115528262011304, 'num_leaves': 74, 'max_depth': 5, 'subsample': 0.7051612900030495, 'colsample_bytree': 0.7218630022794463, 'reg_alpha': 0.004969816141554229, 'reg_lambda': 7.501208085675168}. Best is trial 0 with value: 0.736593602880274.
<ipython-input-43-980fb401a53f>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
<ipython-input-43-980fb401a53f>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10.0),
<ip

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.093246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 772
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521


[I 2025-02-14 08:25:23,450] Trial 1 finished with value: 0.7378716379456746 and parameters: {'n_estimators': 390, 'learning_rate': 0.009554906099827534, 'num_leaves': 45, 'max_depth': 9, 'subsample': 0.6024047112532847, 'colsample_bytree': 0.8738135810014938, 'reg_alpha': 0.7680968170476943, 'reg_lambda': 3.462299928002031}. Best is trial 1 with value: 0.7378716379456746.
<ipython-input-43-980fb401a53f>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
<ipython-input-43-980fb401a53f>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10.0),
<ip

...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.071971 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 772
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521


[I 2025-02-14 08:37:05,621] Trial 28 finished with value: 0.7394611124336161 and parameters: {'n_estimators': 645, 'learning_rate': 0.015863375766163326, 'num_leaves': 38, 'max_depth': 10, 'subsample': 0.7826540907619437, 'colsample_bytree': 0.5816354202896985, 'reg_alpha': 1.873163662490265, 'reg_lambda': 0.009615467649695434}. Best is trial 28 with value: 0.7394611124336161.
<ipython-input-43-980fb401a53f>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
<ipython-input-43-980fb401a53f>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10.0)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.075270 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 772
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[I 2025-02-14 08:37:30,726] Trial 29 finished with value: 0.738631055021191 and parameters: {'n_estimators': 634, 'learning_rate': 0.012935043901286797, 'num_leaves': 47, 'max_depth': 5, 'subsample': 0.7588986519324091, 'colsample_bytree': 0.681427838025344, 'reg_alpha': 2.5779136825725666, 'reg_lambda': 0.6045044047514422}. Best is trial 28 with value: 0.7394611124336161.


🔹 Best Params: {'n_estimators': 645, 'learning_rate': 0.015863375766163326, 'num_leaves': 38, 'max_depth': 10, 'subsample': 0.7826540907619437, 'colsample_bytree': 0.5816354202896985, 'reg_alpha': 1.873163662490265, 'reg_lambda': 0.009615467649695434}


In [ ]:
# 최적 하이퍼파라미터를 적용하여 모델 학습
lgbm_best = LGBMClassifier(**best_params)
lgbm_best.fit(X_train, y_train)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 53102, number of negative: 151978
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.135979 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 772
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 60
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051521
[LightGBM] [Info] Start training from score -1.051521


LGBMClassifier(colsample_bytree=0.5816354202896985,
               learning_rate=0.015863375766163326, max_depth=10,
               n_estimators=645, num_leaves=38, reg_alpha=1.873163662490265,
               reg_lambda=0.009615467649695434, subsample=0.7826540907619437)

In [ ]:
# 검증 데이터 평가 (Validation ROC-AUC)
y_val_pred = lgbm_best.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_val_pred)
print('Validation ROC-AUC: {roc_auc:.4f}')


✅ Validation ROC-AUC: 0.7394


In [ ]:
# 테스트 데이터 예측
test_preds = lgbm_best.predict_proba(df_test)[:, 1]


In [ ]:
# 제출 파일 생성
submission = pd.DataFrame({'ID': df_test_ids, 'probability': test_preds})
submission.to_csv('submission_20.csv', index=False)
print("Submission file saved: submission_20.csv")

✅ Submission file saved: submission_20.csv
